In [1]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv('../execution_results2/execute_sw.csv')


In [2]:
df.head()

,Instance,Algorithme,Seed,Score,Lambda,MaxEval,CPU-Used-Time (ms)
0,instances/atsp_rand_50_75.txt,sampled_walk_swap,50,1738,5,1000000,1973.446
1,instances/atsp_rand_50_75.txt,sampled_walk_swap,50,1719,9,1000000,1160.193
2,instances/atsp_rand_50_75.txt,sampled_walk_swap,51,1706,5,1000000,1971.424
3,instances/atsp_rand_50_75.txt,sampled_walk_swap,51,1717,9,1000000,1160.676
4,instances/atsp_rand_50_75.txt,sampled_walk_swap,52,1751,5,1000000,2032.618


In [3]:
neighbor_relations = ["swap","2opt"]
versus = ('swap', '2opt')
instances = df['Instance'].unique()
lambdas = df["Lambda"].unique()


In [4]:
def get_score(instance,neighbor_relation,seed,ld):
    filtre = (
    (df['Instance'] == instance) &
    (df['Algorithme'] == f"sampled_walk_{neighbor_relation}") &
    (df['Seed'] == seed) & 
    (df["Lambda"] == ld)
    )
    r=df[filtre]
    if r.size != 0:
        return r.iloc[0][3]
    else:
        print(instance,neighbor_relation,seed)
    

In [5]:
def gagne(instance,seed, vs,ld):
    #print("vs:",vs)
    s1 = get_score(instance,vs[0],seed,ld)
    s2 = get_score(instance,vs[1],seed,ld)
    return s1 < s2

In [15]:
get_score("instances/atsp_rand_50_75.txt","swap",50,9)

1719

In [13]:
get_score("instances/atsp_rand_50_75.txt","2opt",50,9)

2034

In [8]:
gagne("instances/atsp_rand_50_75.txt",50, ("swap","2opt"),9)

True

In [9]:
result_tuple={"instances" : instances, "swap_vs_2opt" : []}
for instance in instances:
    
    r=[0,0]
    for ld in lambdas:
        for seed in df["Seed"].unique():
            if gagne(instance,seed, versus,ld):
                r[0]+=1
            else:
                r[1]+=1
            
    result_tuple["swap_vs_2opt"].append((r[0],r[1]))
            
            #print(r)
                
result_tuple = pd.DataFrame(result_tuple)
result_tuple

,instances,swap_vs_2opt
0,instances/atsp_rand_50_75.txt,"(20, 0)"
1,instances/atsp_rand_50_100.txt,"(20, 0)"
2,instances/atsp_rand_60_50.txt,"(20, 0)"
3,instances/atsp_rand_60_100.txt,"(20, 0)"
4,instances/atsp_rand_70_55.txt,"(20, 0)"
5,instances/atsp_rand_70_100.txt,"(20, 0)"
6,instances/atsp_rand_80_70.txt,"(20, 0)"
7,instances/atsp_rand_80_100.txt,"(20, 0)"


In [10]:
df_group = df[["Algorithme","Score"]].groupby("Algorithme")
df_group.mean()

,Score
Algorithme,
sampled_walk_2opt,1818.96875
sampled_walk_swap,1209.74375


In [16]:
df_group = df[["Algorithme","CPU-Used-Time (ms)"]].groupby("Algorithme")
df_group.mean()

,CPU-Used-Time (ms)
Algorithme,
sampled_walk_2opt,2633.410887
sampled_walk_swap,2666.000194
